In [8]:
import json
import pandas as pd
import numpy as np
import sys

pd.set_option('display.max_colwidth', None)
sys.path.append('./src-py')

In [9]:
import sbert_training
from sklearn.metrics import precision_recall_fscore_support

In [10]:
from sentence_transformers import SentenceTransformer, InputExample, LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, BinaryClassificationEvaluator
from torch.utils.data import DataLoader
from sentence_transformers.evaluation import TripletEvaluator
from datetime import datetime
from sentence_transformers import util
from zipfile import ZipFile
from sentence_transformers.datasets import SentenceLabelDataset
from sentence_transformers.datasets import NoDuplicatesDataLoader

import logging

logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
logger = logging.getLogger(__name__)

In [11]:
taska_training_df = pd.read_csv('../data/TaskA_train.csv')
taskb_training_df = pd.read_csv('../data/TaskB_train.csv')

taska_valid_df = pd.read_csv('../data/TaskA_dev.csv')
taskb_valid_df = pd.read_csv('../data/TaskB_dev.csv')

In [12]:
#Mapping labels
taska_training_df = taska_training_df[taska_training_df.Novelty != 0]
taska_valid_df    = taska_valid_df[taska_valid_df.Novelty != 0]

taska_training_df['label'] = taska_training_df.Novelty.apply(lambda x : 1 if x == 1 else 0)
taska_valid_df['label'] = taska_valid_df.Novelty.apply(lambda x : 1 if x == 1 else 0)

taska_training_df['Premise'] = taska_training_df.apply(lambda x: x['topic'] + ' : ' +  x['Premise'], axis=1)
taska_valid_df['Premise'] = taska_valid_df.apply(lambda x: x['topic'] + ' : ' +  x['Premise'], axis=1)

In [13]:
taska_training_df['input_txt'] = taska_training_df.apply(lambda x: '[CLS] {} [SEP] {} [SEP] {} [SEP]'.format(x['topic'], x['Premise'], x['Conclusion']), axis=1)

In [16]:
taska_training_df[['Premise', 'Conclusion', 'Novelty', 'label']].head()

,Premise,Conclusion,Novelty,label
0,"TV viewing is harmful to children : The popularity of TV watching is among the reasons of this phenomenon. Violence, aggression, crimes and wars are broadcast through the daily news as well as in movies, showing dark pictures that encourage psychological tension, pessimism and negative emotions.",Depression is a well-known psychological problem of modern society that is partly caused by TV watching:,1,1
1,"TV viewing is harmful to children : The popularity of TV watching is among the reasons of this phenomenon. Violence, aggression, crimes and wars are broadcast through the daily news as well as in movies, showing dark pictures that encourage psychological tension, pessimism and negative emotions.",Children's TV viewing fosters negative emotions,-1,0
2,"TV viewing is harmful to children : The popularity of TV watching is among the reasons of this phenomenon. Violence, aggression, crimes and wars are broadcast through the daily news as well as in movies, showing dark pictures that encourage psychological tension, pessimism and negative emotions.",Popularity of TV is harmful to children,1,1
3,"TV viewing is harmful to children : The popularity of TV watching is among the reasons of this phenomenon. Violence, aggression, crimes and wars are broadcast through the daily news as well as in movies, showing dark pictures that encourage psychological tension, pessimism and negative emotions.",Violence on TV and in movies encourages psychological stress,1,1
4,"TV viewing is harmful to children : The popularity of TV watching is among the reasons of this phenomenon. Violence, aggression, crimes and wars are broadcast through the daily news as well as in movies, showing dark pictures that encourage psychological tension, pessimism and negative emotions.",US-India deal does not cap or limit Indian fissile material production.,-1,0


In [8]:
taska_training_df.label.value_counts()

1    401
0    320
Name: label, dtype: int64

In [9]:
taska_valid_df.label.value_counts()

1    125
0     74
Name: label, dtype: int64

In [17]:
def get_training_examples(df, eval_df, loss):
    
    logger.info("Read Triplet train dataset")
    train_examples = []
    for idx, row in df.iterrows():
        if loss == 'ContrastiveLoss':
            train_examples.append(InputExample(texts=[row['Premise'], row['Conclusion']], label=row['label']))
        elif loss == 'MultipleNegativesRankingLoss':
            if row['label'] == 1:
                train_examples.append(InputExample(texts=[row['Premise'], row['Conclusion']], label=1))
        else:
            train_examples.append(InputExample(texts=[row['anchor'], row['pos'], row['neg']], label=0))
            
    
    dev_samples = []
    for idx, row in eval_df.iterrows():
        dev_samples.append(InputExample(texts=[row['Premise'], row['Conclusion']], label=row['label']))
    
    return train_examples, dev_samples

            
def train_model(df, eval_df, output_path, model_name, num_epochs=3, train_batch_size=16, model_suffix='', \
                data_file_suffix='', max_seq_length=256, 
                special_tokens=[], loss='Triplet', sentence_transformer=False, evaluation_steps=5):
    
    output_path = output_path + model_name+ "-" + model_suffix + "-"+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

    if sentence_transformer:
        word_embedding_model = SentenceTransformer(model_name)
        word_embedding_model.max_seq_length = max_seq_length
    else:
        word_embedding_model = models.Transformer(model_name)
        word_embedding_model.max_seq_length = max_seq_length
    
    
    if len(special_tokens) > 0:
        word_embedding_model.tokenizer.add_tokens(special_tokens, special_tokens=True)
        word_embedding_model.resize_token_embeddings(len(word_embedding_model.tokenizer))
        
    # Apply mean pooling to get one fixed sized sentence vector
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                                   pooling_mode_mean_tokens=True,
                                   pooling_mode_cls_token=False,
                                   pooling_mode_max_tokens=False)

    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])


    train_examples, dev_samples = get_training_examples(df, eval_df, loss)

    print('Len of training: {}'.format(len(train_examples)))
    print('Len of Dev: {}'.format(len(dev_samples)))
    
    if loss == 'MultipleNegativesRankingLoss':
        # Special data loader that avoid duplicates within a batch
        train_dataloader = NoDuplicatesDataLoader(train_examples, batch_size=train_batch_size)
        # Our training loss
        train_loss = losses.MultipleNegativesRankingLoss(model)
    elif loss == 'ContrastiveLoss':
        train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=train_batch_size)
        train_loss = losses.ContrastiveLoss(model)
    else:
        train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=train_batch_size)
        train_loss = losses.TripletLoss(model)
    

    evaluator = BinaryClassificationEvaluator.from_input_examples(dev_samples, batch_size=train_batch_size, name='sts-dev')

    warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data

    print('Evaluating before start learning.....')
    model.evaluate(evaluator)
    print('====== Start Training =======')
    
    # Train the model
    model.fit(train_objectives=[(train_dataloader, train_loss)],
              evaluator=evaluator,
              epochs=num_epochs,
              save_best_model=True,
              checkpoint_save_steps=evaluation_steps,
              optimizer_params={'lr':5e-06},
              checkpoint_save_total_limit=3,
              evaluation_steps=evaluation_steps,
              warmup_steps=warmup_steps,
              output_path=output_path)
    
    return model, evaluator

In [14]:
#train_model(taska_training_df, taska_valid_df, '../data/output/', 'sentence-transformers/nli-roberta-large', num_epochs=20, train_batch_size=16, model_suffix='', data_file_suffix='', max_seq_length=256, special_tokens=[], loss='ContrastiveLoss', sentence_transformer=False)

In [18]:
trained_model, evaluator = train_model(taska_training_df, taska_valid_df, '../data/output/', 
            'sentence-transformers/nli-roberta-large', 
            num_epochs=15, train_batch_size=32,
            model_suffix='ranking-loss', max_seq_length=512, special_tokens=[], 
            loss='MultipleNegativesRankingLoss', sentence_transformer=False, evaluation_steps=10)

2022-07-01 14:24:14 - Use pytorch device: cuda
2022-07-01 14:24:14 - Read Triplet train dataset
Len of training: 401
Len of Dev: 199
Evaluating before start learning.....
2022-07-01 14:24:14 - Binary Accuracy Evaluation of the model on sts-dev dataset:
2022-07-01 14:24:14 - Accuracy with Cosine-Similarity:           72.36	(Threshold: 0.6503)
2022-07-01 14:24:14 - F1 with Cosine-Similarity:                 80.69	(Threshold: 0.5985)
2022-07-01 14:24:14 - Precision with Cosine-Similarity:          70.91
2022-07-01 14:24:14 - Recall with Cosine-Similarity:             93.60
2022-07-01 14:24:14 - Average Precision with Cosine-Similarity:  82.54

2022-07-01 14:24:14 - Accuracy with Manhatten-Distance:           72.36	(Threshold: 646.3732)
2022-07-01 14:24:14 - F1 with Manhatten-Distance:                 80.56	(Threshold: 679.7071)
2022-07-01 14:24:14 - Precision with Manhatten-Distance:          71.17
2022-07-01 14:24:14 - Recall with Manhatten-Distance:             92.80
2022-07-01 14:24:14

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Iteration:   0%|          | 0/12 [00:00<?, ?it/s]

2022-07-01 14:24:16 - Binary Accuracy Evaluation of the model on sts-dev dataset in epoch 0 after 10 steps:
2022-07-01 14:24:16 - Accuracy with Cosine-Similarity:           72.36	(Threshold: 0.6591)
2022-07-01 14:24:16 - F1 with Cosine-Similarity:                 80.56	(Threshold: 0.6068)
2022-07-01 14:24:16 - Precision with Cosine-Similarity:          71.17
2022-07-01 14:24:16 - Recall with Cosine-Similarity:             92.80
2022-07-01 14:24:16 - Average Precision with Cosine-Similarity:  82.88

2022-07-01 14:24:16 - Accuracy with Manhatten-Distance:           72.86	(Threshold: 635.0920)
2022-07-01 14:24:16 - F1 with Manhatten-Distance:                 80.56	(Threshold: 672.8419)
2022-07-01 14:24:16 - Precision with Manhatten-Distance:          71.17
2022-07-01 14:24:16 - Recall with Manhatten-Distance:             92.80
2022-07-01 14:24:16 - Average Precision with Manhatten-Distance:  82.97

2022-07-01 14:24:16 - Accuracy with Euclidean-Distance:           72.86	(Threshold: 25.3105

Iteration:   0%|          | 0/12 [00:00<?, ?it/s]

...

2022-07-01 14:24:56 - Binary Accuracy Evaluation of the model on sts-dev dataset in epoch 12 after 10 steps:
2022-07-01 14:24:57 - Accuracy with Cosine-Similarity:           69.85	(Threshold: 0.6294)
2022-07-01 14:24:57 - F1 with Cosine-Similarity:                 79.11	(Threshold: 0.4138)
2022-07-01 14:24:57 - Precision with Cosine-Similarity:          65.45
2022-07-01 14:24:57 - Recall with Cosine-Similarity:             100.00
2022-07-01 14:24:57 - Average Precision with Cosine-Similarity:  81.79

2022-07-01 14:24:57 - Accuracy with Manhatten-Distance:           70.85	(Threshold: 652.7652)
2022-07-01 14:24:57 - F1 with Manhatten-Distance:                 79.22	(Threshold: 746.9213)
2022-07-01 14:24:57 - Precision with Manhatten-Distance:          66.67
2022-07-01 14:24:57 - Recall with Manhatten-Distance:             97.60
2022-07-01 14:24:57 - Average Precision with Manhatten-Distance:  81.87

2022-07-01 14:24:57 - Accuracy with Euclidean-Distance:           68.34	(Threshold: 25.55

Iteration:   0%|          | 0/12 [00:00<?, ?it/s]

2022-07-01 14:24:59 - Binary Accuracy Evaluation of the model on sts-dev dataset in epoch 13 after 10 steps:
2022-07-01 14:24:59 - Accuracy with Cosine-Similarity:           69.85	(Threshold: 0.6289)
2022-07-01 14:24:59 - F1 with Cosine-Similarity:                 79.11	(Threshold: 0.4121)
2022-07-01 14:24:59 - Precision with Cosine-Similarity:          65.45
2022-07-01 14:24:59 - Recall with Cosine-Similarity:             100.00
2022-07-01 14:24:59 - Average Precision with Cosine-Similarity:  81.70

2022-07-01 14:24:59 - Accuracy with Manhatten-Distance:           70.85	(Threshold: 653.0719)
2022-07-01 14:24:59 - F1 with Manhatten-Distance:                 79.22	(Threshold: 747.4995)
2022-07-01 14:24:59 - Precision with Manhatten-Distance:          66.67
2022-07-01 14:24:59 - Recall with Manhatten-Distance:             97.60
2022-07-01 14:24:59 - Average Precision with Manhatten-Distance:  81.87

2022-07-01 14:24:59 - Accuracy with Euclidean-Distance:           68.34	(Threshold: 25.59

Iteration:   0%|          | 0/12 [00:00<?, ?it/s]

2022-07-01 14:25:02 - Binary Accuracy Evaluation of the model on sts-dev dataset in epoch 14 after 10 steps:
2022-07-01 14:25:02 - Accuracy with Cosine-Similarity:           69.35	(Threshold: 0.6377)
2022-07-01 14:25:02 - F1 with Cosine-Similarity:                 79.11	(Threshold: 0.4116)
2022-07-01 14:25:02 - Precision with Cosine-Similarity:          65.45
2022-07-01 14:25:02 - Recall with Cosine-Similarity:             100.00
2022-07-01 14:25:02 - Average Precision with Cosine-Similarity:  81.70

2022-07-01 14:25:02 - Accuracy with Manhatten-Distance:           70.85	(Threshold: 653.2681)
2022-07-01 14:25:02 - F1 with Manhatten-Distance:                 79.22	(Threshold: 747.7288)
2022-07-01 14:25:02 - Precision with Manhatten-Distance:          66.67
2022-07-01 14:25:02 - Recall with Manhatten-Distance:             97.60
2022-07-01 14:25:02 - Average Precision with Manhatten-Distance:  81.85

2022-07-01 14:25:02 - Accuracy with Euclidean-Distance:           68.34	(Threshold: 25.60

### Using the auto-generated conclusions:

In [12]:
taska_training_df = pd.read_pickle('../data/TaskA_train_with_extra_conclusions.pkl')
taska_valid_df = pd.read_csv('../data/TaskA_dev.csv')

In [13]:
#Mapping labels
taska_training_df = taska_training_df[taska_training_df.Validity != 0]
taska_valid_df    = taska_valid_df[taska_valid_df.Validity != 0]

taska_training_df['label'] = taska_training_df.Validity.apply(lambda x : 1 if x == 1 else 0)
taska_valid_df['label'] = taska_valid_df.Validity.apply(lambda x : 1 if x == 1 else 0)

taska_training_df['Premise'] = taska_training_df.apply(lambda x: x['topic'] + ' : ' +  x['Premise'], axis=1)
taska_valid_df['Premise'] = taska_valid_df.apply(lambda x: x['topic'] + ' : ' +  x['Premise'], axis=1)

In [19]:
train_model(taska_training_df, taska_valid_df, '../data/output/', 
            'sentence-transformers/nli-roberta-large', 
            num_epochs=15, train_batch_size=32,
            model_suffix='ranking-loss-extra-data', max_seq_length=512, special_tokens=[], 
            loss='MultipleNegativesRankingLoss', sentence_transformer=False, evaluation_steps=10)

2022-07-01 14:27:28 - Use pytorch device: cuda
2022-07-01 14:27:28 - Read Triplet train dataset
Len of training: 401
Len of Dev: 199
Evaluating before start learning.....
2022-07-01 14:27:28 - Binary Accuracy Evaluation of the model on sts-dev dataset:
2022-07-01 14:27:28 - Accuracy with Cosine-Similarity:           72.36	(Threshold: 0.6503)
2022-07-01 14:27:28 - F1 with Cosine-Similarity:                 80.69	(Threshold: 0.5985)
2022-07-01 14:27:28 - Precision with Cosine-Similarity:          70.91
2022-07-01 14:27:28 - Recall with Cosine-Similarity:             93.60
2022-07-01 14:27:28 - Average Precision with Cosine-Similarity:  82.54

2022-07-01 14:27:28 - Accuracy with Manhatten-Distance:           72.36	(Threshold: 646.3732)
2022-07-01 14:27:28 - F1 with Manhatten-Distance:                 80.56	(Threshold: 679.7071)
2022-07-01 14:27:28 - Precision with Manhatten-Distance:          71.17
2022-07-01 14:27:28 - Recall with Manhatten-Distance:             92.80
2022-07-01 14:27:28

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Iteration:   0%|          | 0/12 [00:00<?, ?it/s]

2022-07-01 14:27:30 - Binary Accuracy Evaluation of the model on sts-dev dataset in epoch 0 after 10 steps:
2022-07-01 14:27:30 - Accuracy with Cosine-Similarity:           73.37	(Threshold: 0.6399)
2022-07-01 14:27:30 - F1 with Cosine-Similarity:                 80.59	(Threshold: 0.6399)
2022-07-01 14:27:30 - Precision with Cosine-Similarity:          74.32
2022-07-01 14:27:30 - Recall with Cosine-Similarity:             88.00
2022-07-01 14:27:30 - Average Precision with Cosine-Similarity:  83.01

2022-07-01 14:27:30 - Accuracy with Manhatten-Distance:           72.86	(Threshold: 634.2710)
2022-07-01 14:27:30 - F1 with Manhatten-Distance:                 80.57	(Threshold: 662.9831)
2022-07-01 14:27:30 - Precision with Manhatten-Distance:          72.15
2022-07-01 14:27:30 - Recall with Manhatten-Distance:             91.20
2022-07-01 14:27:30 - Average Precision with Manhatten-Distance:  82.94

2022-07-01 14:27:30 - Accuracy with Euclidean-Distance:           72.86	(Threshold: 25.3020

Iteration:   0%|          | 0/12 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/12 [00:00<?, ?it/s]

2022-07-01 14:28:16 - Binary Accuracy Evaluation of the model on sts-dev dataset in epoch 13 after 10 steps:
2022-07-01 14:28:16 - Accuracy with Cosine-Similarity:           68.34	(Threshold: 0.6423)
2022-07-01 14:28:16 - F1 with Cosine-Similarity:                 79.37	(Threshold: 0.4397)
2022-07-01 14:28:16 - Precision with Cosine-Similarity:          65.79
2022-07-01 14:28:16 - Recall with Cosine-Similarity:             100.00
2022-07-01 14:28:16 - Average Precision with Cosine-Similarity:  81.57

2022-07-01 14:28:16 - Accuracy with Manhatten-Distance:           70.85	(Threshold: 645.9094)
2022-07-01 14:28:16 - F1 with Manhatten-Distance:                 79.22	(Threshold: 753.9757)
2022-07-01 14:28:16 - Precision with Manhatten-Distance:          66.67
2022-07-01 14:28:16 - Recall with Manhatten-Distance:             97.60
2022-07-01 14:28:16 - Average Precision with Manhatten-Distance:  82.10

2022-07-01 14:28:16 - Accuracy with Euclidean-Distance:           68.84	(Threshold: 25.60

Iteration:   0%|          | 0/12 [00:00<?, ?it/s]

2022-07-01 14:28:19 - Binary Accuracy Evaluation of the model on sts-dev dataset in epoch 14 after 10 steps:
2022-07-01 14:28:19 - Accuracy with Cosine-Similarity:           68.34	(Threshold: 0.6422)
2022-07-01 14:28:19 - F1 with Cosine-Similarity:                 79.37	(Threshold: 0.4395)
2022-07-01 14:28:19 - Precision with Cosine-Similarity:          65.79
2022-07-01 14:28:19 - Recall with Cosine-Similarity:             100.00
2022-07-01 14:28:19 - Average Precision with Cosine-Similarity:  81.55

2022-07-01 14:28:19 - Accuracy with Manhatten-Distance:           70.85	(Threshold: 646.4954)
2022-07-01 14:28:19 - F1 with Manhatten-Distance:                 79.22	(Threshold: 754.0268)
2022-07-01 14:28:19 - Precision with Manhatten-Distance:          66.67
2022-07-01 14:28:19 - Recall with Manhatten-Distance:             97.60
2022-07-01 14:28:19 - Average Precision with Manhatten-Distance:  82.11

2022-07-01 14:28:19 - Accuracy with Euclidean-Distance:           68.84	(Threshold: 25.62

(SentenceTransformer(
   (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: RobertaModel 
   (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
 ),
 <sentence_transformers.evaluation.BinaryClassificationEvaluator.BinaryClassificationEvaluator at 0x7ff178ede910>)

#### Generating predictions for argument validity:

- Load best model and use it to predict the labels for the eval_df.
- Best model is: nli-roberta-large-ranking-loss-extra-data-2022-07-01_14-27-18

In [43]:
taska_valid_df = pd.read_csv('../data/TaskA_dev.csv')
taska_valid_df = taska_valid_df[taska_valid_df.Validity != 0]

In [47]:
def predict_labels(df, model_path, clm1, clm2, threshold):
    best_model = SentenceTransformer(model_path)

    encoded_premises = best_model.encode(taska_valid_df[clm1].tolist())
    encoded_conclusions = best_model.encode(taska_valid_df[clm2].tolist())
    
    scores = [util.pytorch_cos_sim(x[0], x[1]).item() for x in zip(encoded_conclusions, encoded_premises)]
    labels = [1 if x > threshold else -1 for x in scores]
    
    df['pred_labels'] = labels
    
    return df

In [58]:
eval_df = predict_labels(taska_valid_df, '../data/output/sentence-transformers/nli-roberta-large-ranking-loss-extra-data-2022-07-01_14-27-18/', 'Premise', 'Conclusion', 0.6146)
#eval_df = predict_labels(taska_valid_df, '../data/output/sentence-transformers/nli-roberta-large-ranking-loss-2022-07-01_14-24-04', 'Premise', 'Conclusion', 0.6146)

2022-07-01 15:09:48 - Load pretrained SentenceTransformer: ../data/output/sentence-transformers/nli-roberta-large-ranking-loss-extra-data-2022-07-01_14-27-18/
2022-07-01 15:09:50 - Use pytorch device: cuda


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

In [59]:
precision, recall, f1, _ = precision_recall_fscore_support(eval_df.Validity.tolist(), eval_df.pred_labels.tolist(), average='binary')

print('Precision: {}, Recall {}, F1: {}'.format(precision, recall, f1))

Precision: 0.7, Recall 0.896, F1: 0.7859649122807016
